# সহজ ইমেজ শ্রেণীবিন্যাসকারী

এই নোটবুকটি আপনাকে একটি পূর্ব-প্রশিক্ষিত নিউরাল নেটওয়ার্ক ব্যবহার করে ইমেজ শ্রেণীবিন্যাস করতে শেখাবে।

**আপনি যা শিখবেন:**
- কীভাবে একটি পূর্ব-প্রশিক্ষিত মডেল লোড এবং ব্যবহার করবেন
- ইমেজ প্রিপ্রসেসিং
- ইমেজে পূর্বাভাস তৈরি করা
- আত্মবিশ্বাস স্কোর বোঝা

**ব্যবহার ক্ষেত্র:** ইমেজে বস্তু সনাক্ত করা (যেমন "বিড়াল", "কুকুর", "গাড়ি" ইত্যাদি)


## ধাপ ১: প্রয়োজনীয় লাইব্রেরি আমদানি করুন

চলুন আমরা যেসব টুল দরকার সেগুলো আমদানি করি। যদি সবকিছু এখনই বুঝতে না পারেন, চিন্তা করবেন না!


In [ ]:
# Core libraries
import numpy as np
from PIL import Image
import requests
from io import BytesIO

# TensorFlow for deep learning
try:
    import tensorflow as tf
    from tensorflow.keras.applications import MobileNetV2
    from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
    print("✅ TensorFlow loaded successfully!")
    print(f"   Version: {tf.__version__}")
except ImportError:
    print("❌ Please install TensorFlow: pip install tensorflow")

## ধাপ ২: প্রি-ট্রেইনড মডেল লোড করুন

আমরা **MobileNetV2** ব্যবহার করব, একটি নিউরাল নেটওয়ার্ক যা ইতিমধ্যে লক্ষ লক্ষ ছবির উপর প্রশিক্ষিত।

এটি **ট্রান্সফার লার্নিং** নামে পরিচিত - অন্য কেউ প্রশিক্ষিত একটি মডেল ব্যবহার করা!


In [ ]:
print("📦 Loading pre-trained MobileNetV2 model...")
print("   This may take a minute on first run (downloading weights)...")

# Load the model
# include_top=True means we use the classification layer
# weights='imagenet' means it was trained on ImageNet dataset
model = MobileNetV2(weights='imagenet', include_top=True)

print("✅ Model loaded!")
print(f"   The model can recognize 1000 different object categories")

## ধাপ ৩: সহায়ক ফাংশনসমূহ

চলুন আমাদের মডেলের জন্য ছবি লোড এবং প্রস্তুত করার ফাংশন তৈরি করি।


In [ ]:
def load_image_from_url(url):
    """
    Load an image from a URL.
    
    Args:
        url: Web address of the image
        
    Returns:
        PIL Image object
    """
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    return img


def prepare_image(img):
    """
    Prepare an image for the model.
    
    Steps:
    1. Resize to 224x224 (model's expected size)
    2. Convert to array
    3. Add batch dimension
    4. Preprocess for MobileNetV2
    
    Args:
        img: PIL Image
        
    Returns:
        Preprocessed image array
    """
    # Resize to 224x224 pixels
    img = img.resize((224, 224))
    
    # Convert to numpy array
    img_array = np.array(img)
    
    # Add batch dimension (model expects multiple images)
    img_array = np.expand_dims(img_array, axis=0)
    
    # Preprocess for MobileNetV2
    img_array = preprocess_input(img_array)
    
    return img_array


def classify_image(img):
    """
    Classify an image and return top predictions.
    
    Args:
        img: PIL Image
        
    Returns:
        List of (class_name, confidence) tuples
    """
    # Prepare the image
    img_array = prepare_image(img)
    
    # Make prediction
    predictions = model.predict(img_array, verbose=0)
    
    # Decode predictions to human-readable labels
    # top=5 means we get the top 5 most likely classes
    decoded = decode_predictions(predictions, top=5)[0]
    
    # Convert to simpler format
    results = [(label, float(confidence)) for (_, label, confidence) in decoded]
    
    return results


print("✅ Helper functions ready!")

## ধাপ ৪: নমুনা ছবিতে পরীক্ষা করুন

চলুন ইন্টারনেট থেকে কিছু ছবি শ্রেণীবদ্ধ করার চেষ্টা করি!


In [ ]:
# Sample images to classify
# These are from Unsplash (free stock photos)
test_images = [
    {
        "url": "https://images.unsplash.com/photo-1514888286974-6c03e2ca1dba?w=400",
        "description": "A cat"
    },
    {
        "url": "https://images.unsplash.com/photo-1552053831-71594a27632d?w=400",
        "description": "A dog"
    },
    {
        "url": "https://images.unsplash.com/photo-1511919884226-fd3cad34687c?w=400",
        "description": "A car"
    },
]

print(f"🧪 Testing on {len(test_images)} images...")
print("=" * 70)

### প্রতিটি চিত্র শ্রেণীবদ্ধ করুন


In [ ]:
for i, img_data in enumerate(test_images, 1):
    print(f"\n📸 Image {i}: {img_data['description']}")
    print("-" * 70)
    
    try:
        # Load image
        img = load_image_from_url(img_data['url'])
        
        # Display image
        display(img.resize((200, 200)))  # Show smaller version
        
        # Classify
        results = classify_image(img)
        
        # Show predictions
        print("\n🎯 Top 5 Predictions:")
        for rank, (label, confidence) in enumerate(results, 1):
            # Create a visual bar
            bar_length = int(confidence * 50)
            bar = "█" * bar_length
            
            print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
        
    except Exception as e:
        print(f"❌ Error: {e}")

print("\n" + "=" * 70)

## ধাপ ৫: আপনার নিজের ছবি চেষ্টা করুন!

নীচের URLটি পরিবর্তন করুন এবং যেকোনো ছবি URL ব্যবহার করুন যা আপনি শ্রেণীবদ্ধ করতে চান।


In [ ]:
# Try your own image!
# Replace this URL with any image URL
custom_image_url = "https://images.unsplash.com/photo-1472491235688-bdc81a63246e?w=400"  # A flower

print("🖼️  Classifying your custom image...")
print("=" * 70)

try:
    # Load and show image
    img = load_image_from_url(custom_image_url)
    display(img.resize((300, 300)))
    
    # Classify
    results = classify_image(img)
    
    # Show results
    print("\n🎯 Top 5 Predictions:")
    print("-" * 70)
    for rank, (label, confidence) in enumerate(results, 1):
        bar_length = int(confidence * 50)
        bar = "█" * bar_length
        print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
    
    # Highlight top prediction
    top_label, top_confidence = results[0]
    print("\n" + "=" * 70)
    print(f"\n🏆 Best guess: {top_label} ({top_confidence*100:.2f}% confident)")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("   Make sure the URL points to a valid image!")

## 💡 কী ঘটল?

1. **আমরা একটি প্রি-ট্রেইনড মডেল লোড করেছি** - MobileNetV2 লক্ষ লক্ষ ছবির উপর প্রশিক্ষিত হয়েছে  
2. **আমরা ছবিগুলো প্রি-প্রসেস করেছি** - মডেলের জন্য সাইজ পরিবর্তন এবং ফরম্যাটিং করেছি  
3. **মডেল ভবিষ্যদ্বাণী করেছে** - এটি ১০০০টি অবজেক্ট ক্লাসের জন্য সম্ভাব্যতা আউটপুট করেছে  
4. **আমরা ফলাফল ডিকোড করেছি** - সংখ্যাগুলোকে মানুষের জন্য পাঠযোগ্য লেবেলে রূপান্তর করেছি  

### আত্মবিশ্বাস স্কোর বোঝা

- **৯০-১০০%**: খুব আত্মবিশ্বাসী (প্রায় নিশ্চিতভাবে সঠিক)  
- **৭০-৯০%**: আত্মবিশ্বাসী (সম্ভবত সঠিক)  
- **৫০-৭০%**: কিছুটা আত্মবিশ্বাসী (সঠিক হতে পারে)  
- **৫০% এর নিচে**: খুব বেশি আত্মবিশ্বাসী নয় (অনিশ্চিত)  

### কেন ভবিষ্যদ্বাণী ভুল হতে পারে?

- **অস্বাভাবিক কোণ বা আলো** - মডেল সাধারণ ছবির উপর প্রশিক্ষিত হয়েছে  
- **একাধিক অবজেক্ট** - মডেল একটি প্রধান অবজেক্ট আশা করে  
- **বিরল অবজেক্ট** - মডেল শুধুমাত্র ১০০০টি ক্যাটাগরি জানে  
- **কম মানের ছবি** - ঝাপসা বা পিক্সেলেটেড ছবি বিশ্লেষণ করা কঠিন  


## 🚀 পরবর্তী পদক্ষেপ

1. **বিভিন্ন ছবি চেষ্টা করুন:**
   - [Unsplash](https://unsplash.com) থেকে ছবি খুঁজুন
   - ডান-ক্লিক করুন → "Copy image address" করে URL সংগ্রহ করুন

2. **পরীক্ষা করুন:**
   - বিমূর্ত শিল্পের সাথে কী ঘটে?
   - এটি কি বিভিন্ন কোণ থেকে বস্তু চিনতে পারে?
   - এটি কি একাধিক বস্তু পরিচালনা করতে পারে?

3. **আরও জানুন:**
   - [Computer Vision lessons](../lessons/4-ComputerVision/README.md) অন্বেষণ করুন
   - নিজের ইমেজ ক্লাসিফায়ার প্রশিক্ষণ দিতে শিখুন
   - CNNs (Convolutional Neural Networks) কীভাবে কাজ করে তা বুঝুন

---

## 🎉 অভিনন্দন!

আপনি একটি অত্যাধুনিক নিউরাল নেটওয়ার্ক ব্যবহার করে একটি ইমেজ ক্লাসিফায়ার তৈরি করেছেন!

এই একই প্রযুক্তি শক্তি যোগায়:
- Google Photos (আপনার ছবিগুলো সংগঠিত করা)
- স্বয়ংচালিত গাড়ি (বস্তু চিনতে)
- চিকিৎসা নির্ণয় (এক্স-রে বিশ্লেষণ)
- গুণমান নিয়ন্ত্রণ (ত্রুটি সনাক্ত করা)

অন্বেষণ এবং শেখা চালিয়ে যান! 🚀



---

**অস্বীকৃতি**:  
এই নথিটি AI অনুবাদ পরিষেবা [Co-op Translator](https://github.com/Azure/co-op-translator) ব্যবহার করে অনুবাদ করা হয়েছে। আমরা যথাসাধ্য সঠিকতা নিশ্চিত করার চেষ্টা করি, তবে অনুগ্রহ করে মনে রাখবেন যে স্বয়ংক্রিয় অনুবাদে ত্রুটি বা অসঙ্গতি থাকতে পারে। মূল ভাষায় থাকা নথিটিকে প্রামাণিক উৎস হিসেবে বিবেচনা করা উচিত। গুরুত্বপূর্ণ তথ্যের ক্ষেত্রে, পেশাদার মানব অনুবাদ সুপারিশ করা হয়। এই অনুবাদ ব্যবহারের ফলে কোনো ভুল বোঝাবুঝি বা ভুল ব্যাখ্যা হলে আমরা দায়বদ্ধ থাকব না।
